In [478]:
!pip install pandas numpy matplotlib yfinance backtrader datetime timedelta asyncio 
from IPython.core.display import clear_output
clear_output()

In [479]:
import concurrent.futures

import yfinance as yf
import backtrader as bt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose

In [480]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# Optionally, set the max rows displayed in the output as well
pd.set_option('display.max_rows', 100)

# Introduction
This notebook is designed to showcase a comprehensive analysis of a diverse portfolio, integrating Index Funds, Leveraged ETFs, monthly dividend REITs and ETFs, and quarterly dividend stocks. The analysis includes portfolio beta evaluation, backtesting a Dollar-Cost Averaging (DCA) strategy that optimizes timing and risk management with technical and quantative analysis, and hedged with options pricing.


In [481]:
# Define Portfolio Assets
index_funds = ['SPY', 'QQQ', 'DAX']
leveraged_etfs = ['TQQQ', 'UMDD', 'UDOW', 'SOXL', 'NVDL', 'TSLL']
monthly_dividend_reits_etfs = ['O', 'AGNC', 'CSHI', 'JEPI', 'NUSI']
quarterly_dividend_stocks = [
    'SPYD', 'MSFT', 'INTC', 'F', 'CSCO', 'BAC', 'PFE', 'BX', 'MO', 
    'DOW', 'WMT', 'T', 'KMB', 'SWK', 'IBM', 'PEP', 'KO', 'JNJ'
]
hedging = ['VIX', 'UVXY', 'SPXS' ]

### Beta in Finance

In finance, Beta (β) is a measure of a stock's volatility in relation to the market. It indicates how sensitive the price of a particular stock is to movements in the overall market. Specifically, beta quantifies the systematic risk of a stock, which is the risk that cannot be diversified away because it is inherent to the entire market.

A beta of 1 implies that the stock's price tends to move with the market, while a beta greater than 1 suggests that the stock is more volatile than the market. Conversely, a beta less than 1 indicates that the stock is less volatile than the market.

### Functions Requiring Beta

Several functions in finance require beta as an input parameter. Here's how beta is utilized in each of them:

1. **`get_cost_of_equity(risk_free_rate, beta, market_return)`**:
   - Calculates the cost of equity using the Capital Asset Pricing Model (CAPM), where beta is a key component in assessing the risk of the stock relative to the market.

2. **`calculate_expected_return(risk_free_rate, beta, market_return, market_risk_premium)`**:
   - Utilizes beta within the Capital Asset Pricing Model (CAPM) to compute the expected return of an asset, where beta reflects the systematic risk associated with the asset.

3. **`three_stage_dividend_discount_model(symbol, discount_rate)`**:
   - Involves beta indirectly through `calculate_intrinsic_value`, where beta influences the discount rate used in the valuation.

4. **`residual_income_model(net_income, equity, required_return)`**:
   - Although not directly utilizing beta, the required return on equity (which is part of the model) can be derived from CAPM, where beta plays a pivotal role.

These functions rely on beta to estimate the cost of equity, expected returns, or to indirectly influence the discount rate used in valuation models, as beta serves as a measure of systematic risk inherent in the stock.

### Beta Hedging
Beta hedging is a risk management strategy used by investors to mitigate the impact of market fluctuations on their portfolios. It involves adjusting the beta of the portfolio to align with different market conditions. In this hypothetical scenario, maintaining a portfolio with a beta of 2-3 during bullish cycles implies a higher sensitivity to market movements, potentially leading to greater gains when the market is performing well. Conversely, adjusting the beta to -2 during bearish cycles aims to reduce losses or even profit from market downturns due to the inverse correlation. Transitioning to a beta of 0 during flat cycles indicates a focus on income generation rather than market movements, allowing for stable returns despite stagnant market conditions. By strategically adjusting the beta of the portfolio according to market cycles, investors aim to optimize risk-adjusted returns and better navigate various market environments.


In [482]:
def calculate_beta(stock_symbol, market_symbol='^GSPC', start='2020-01-01', end='2023-01-01'):
    """
    Calculate beta for a given stock symbol relative to a market index, returning the average beta from different methods.

    Args:
        stock_symbol (str): Stock symbol of the company for which beta is to be calculated.
        market_symbol (str): Symbol of the market index. Default is '^GSPC' (S&P 500).
        start (str): Start date for fetching historical data in 'YYYY-MM-DD' format. Default is '2020-01-01'.
        end (str): End date for fetching historical data in 'YYYY-MM-DD' format. Default is '2023-01-01'.

    Returns:
        dict: A dictionary with 'beta' as key and the average beta value as its value, or None if an error occurs.

    Notes:
        - Beta is calculated using different methods: Linear Regression, Covariance Method, Variance Ratio, and scipy's linregress.
        - If an error occurs during data download or calculation, it prints an error message and returns None.
    """
    try:
        stock_data = yf.download(stock_symbol, start=start, end=end)['Adj Close']
        market_data = yf.download(market_symbol, start=start, end=end)['Adj Close']
    except Exception as e:
        print(f'Failed to download data: {str(e)}')
        return None
    
    returns = pd.DataFrame({
        'stock_returns': stock_data.pct_change(),
        'market_returns': market_data.pct_change()
    }).dropna()
    
    beta_values = []
    
    # Linear Regression
    try:
        model = LinearRegression().fit(returns[['market_returns']], returns['stock_returns'])
        beta_values.append(model.coef_[0])
    except Exception as e:
        print(f'Linear Regression error: {str(e)}')
    
    # Covariance Method
    try:
        covariance = returns.cov().iloc[0, 1]
        market_var = returns['market_returns'].var()
        beta_values.append(covariance / market_var)
    except Exception as e:
        print(f'Covariance Method error: {str(e)}')
    
    # Variance Ratio
    try:
        stock_var = returns['stock_returns'].var()
        beta_values.append(stock_var / market_var)
    except Exception as e:
        print(f'Variance Ratio error: {str(e)}')
    
    # Using scipy linregress
    try:
        slope, _, _, _, _ = linregress(returns['market_returns'], returns['stock_returns'])
        beta_values.append(slope)
    except Exception as e:
        print(f'Linregress error: {str(e)}')
    
    # Calculate the average beta if there are any successful beta calculations
    if beta_values:
        average_beta = sum(beta_values) / len(beta_values)
        return {'beta': average_beta}
    else:
        print("Failed to calculate beta using any method.")
        return None


In [483]:
# Combine all asset lists into a single dictionary for easier iteration
assets = {
    'Index Funds': index_funds,
    'LETFS': leveraged_etfs,
    'Monthly Dividend REITs/ETFs': monthly_dividend_reits_etfs,
    'Quarterly Dividend Stocks': quarterly_dividend_stocks,
    'Hedging ETFS': hedging
}

# Initialize a dictionary to store beta values for each asset category
beta_values = {category: {} for category in assets.keys()}

# Iterate through each category and asset to calculate beta values
for category, asset_list in assets.items():
    print(f"\nCalculating beta for {category}:")
    for asset in asset_list:
        try:
            beta = calculate_beta(asset)
            print(f"  {asset}:")
            for method, value in beta.items():
                print(f"    {method}: {value}")
        except Exception as e:
            print(f"  Error calculating beta for {asset}: {str(e)}")
        beta_values[category][asset] = beta

clear_output()

In [484]:
beta_values['Index Funds']

{'SPY': {'beta': 0.9765705664171322},
 'QQQ': {'beta': 1.1546481634216323},
 'DAX': {'beta': 0.9686519831817582}}

In [485]:
beta_values['LETFS']

{'TQQQ': {'beta': 5.361612939954954},
 'UMDD': {'beta': 5.2683784013743065},
 'UDOW': {'beta': 4.151689468094319},
 'SOXL': {'beta': 8.39903364114068},
 'NVDL': {'beta': 6.0982870356498236},
 'TSLL': {'beta': 4.912735916213596}}

In [486]:
beta_values['Monthly Dividend REITs/ETFs']

{'O': {'beta': 1.2556392769010263},
 'AGNC': {'beta': 1.0944188424442447},
 'CSHI': {'beta': 0.01668961977506296},
 'JEPI': {'beta': 0.4944031275287072},
 'NUSI': {'beta': 0.329464536562301}}

In [487]:
beta_values['Quarterly Dividend Stocks']

{'SPYD': {'beta': 1.0803811215837387},
 'MSFT': {'beta': 1.34537203326333},
 'INTC': {'beta': 1.5715480299048292},
 'F': {'beta': 1.8577265952487867},
 'CSCO': {'beta': 1.0816028673278266},
 'BAC': {'beta': 1.5802189592344136},
 'PFE': {'beta': 0.7765915287521632},
 'BX': {'beta': 1.9098830028535865},
 'MO': {'beta': 0.7558236221552064},
 'DOW': {'beta': 1.5726118381518925},
 'WMT': {'beta': 0.6207225136429382},
 'T': {'beta': 0.8231772383478894},
 'KMB': {'beta': 0.5694264660227385},
 'SWK': {'beta': 1.8030842442893962},
 'IBM': {'beta': 0.962003760825742},
 'PEP': {'beta': 0.7985345461110832},
 'KO': {'beta': 0.748428371106244},
 'JNJ': {'beta': 0.5884595155973984}}

In [488]:
beta_values['Hedging ETFS']

{'VIX': {'beta': nan},
 'UVXY': {'beta': 3.1689631835773557},
 'SPXS': {'beta': -0.031338781567867624}}

In [489]:
############################################################################################################
# Define Technical Indicators Functions
############################################################################################################
def bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    data['Bollinger_High'] = rolling_mean + (rolling_std * num_std)
    data['Bollinger_Low'] = rolling_mean - (rolling_std * num_std)
    return data

def macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['Signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

def rsi(data, periods=14, ema=True):
    close_delta = data['Close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    if ema:
        ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
        ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
    else:
        ma_up = up.rolling(window=periods).mean()
        ma_down = down.rolling(window=periods).mean()
    rsi = ma_up / ma_down
    data['RSI'] = 100 - (100 / (1 + rsi))
    return data

def woodie_pivots(data):
    high = data['High']
    low = data['Low']
    close = data['Close']
    pivot = (high + low + 2 * close) / 4
    data['Pivot'] = pivot
    data['R1'] = 2 * pivot - low
    data['S1'] = 2 * pivot - high
    data['R2'] = pivot + (high - low)
    data['S2'] = pivot - (high - low)
    data['R3'] = high + 2 * (pivot - low)
    data['S3'] = low - 2 * (high - pivot)
    data['R4'] = pivot + 3 * (high - low)
    data['S4'] = pivot - 3 * (high - low)
    return data

def obv(data):
    data['OBV'] = np.where(data['Close'] > data['Close'].shift(1), data['Volume'],
                           np.where(data['Close'] < data['Close'].shift(1), -data['Volume'], 0)).cumsum()
    return data

def atr(data, window=14):
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift())
    low_close = np.abs(data['Low'] - data['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    data['ATR'] = true_range.rolling(window=window).mean()
    return data

def stochastic_oscillator(data, window=14):
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()
    data['%K'] = 100 * ((data['Close'] - low_min) / (high_max - low_min))
    data['%D'] = data['%K'].rolling(window=3).mean()
    return data

############################################################################################################
# Process other non-stationary data
############################################################################################################
# Function to detrend time series data using a linear regression model
def detrend_data(data, column):
    # Linear regression model requires reshaped index as a feature
    X = np.arange(len(data)).reshape(-1, 1)
    y = data[column].values  # Original values to detrend
    
    # Create and fit the model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict the trend
    trend = model.predict(X)
    
    # Detrend by subtracting the trend from the original data
    detrended = y - trend
    data[f'{column}_detrended'] = detrended
    
    # Return the detrended data and the trend for further analysis
    return data, trend

def seasonal_decomposition(data, column, period):
    # Perform seasonal decomposition
    decomposition = seasonal_decompose(data[column], model='multiplicative', period=period)
    
    # Add components to DataFrame
    data['trend_component'] = decomposition.trend
    data['seasonal_component'] = decomposition.seasonal
    data['residual_component'] = decomposition.resid
    
    # Seasonally adjust the data
    data[column + '_seasonally_adjusted'] = data[column] / data['seasonal_component']
    
    return data

# Function to calculate price differences
def calculate_price_differences(data, column):
    data[f'{column}_diff'] = data[column].diff()
    return data

# Function to calculate log returns
def calculate_log_returns(data, column):
    data[f'{column}_log_return'] = np.log(data[column] / data[column].shift(1))
    return data

# Function to calculate volume changes
def calculate_volume_changes(data, volume_column):
    data[f'{volume_column}_changes'] = data[volume_column].diff()
    return data



In [490]:

def fetch_fundamentals(ticker):
    """
    Fetches comprehensive fundamental data for a given ticker, including balance sheet and cash flow,
    and returns it as a DataFrame.

    Args:
    - ticker (str): The ticker symbol of the stock.

    Returns:
    - DataFrame: DataFrame with merged market, technical, and fundamental data.
    """
    try:
        # Define start date and end date based on current date and one year ago
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

        ticker_obj = yf.Ticker(ticker)
        
        # Fetch Beta from ticker's info
        beta_value = ticker_obj.info.get('beta', 0)
        
        balance_sheet = ticker_obj.balance_sheet
        cashflow = ticker_obj.cashflow

        balance_sheet_transposed = balance_sheet.T
        cashflow_transposed = cashflow.T

        fundamentals = pd.concat([balance_sheet_transposed, cashflow_transposed], axis=1)
        fundamentals.index.names = ['Date']
        
        # Insert Beta as the first column
        fundamentals.insert(0, 'Beta', beta_value)

        fundamentals.fillna(method='backfill', inplace=True)
        fundamentals.fillna(method='ffill', inplace=True)
        fundamentals.fillna(0, inplace=True)

        # Example of calculating growth rate of free cash flows (replace with your actual data)
        free_cash_flows = pd.Series([100, 120, 140, 160, 180])
        growth_rate = free_cash_flows.pct_change().mean()
        print("Free Cash Flow Growth Rate:", growth_rate)

        return fundamentals

    except Exception as e:
        print(f"Failed to fetch or process fundamental data for {ticker}: {e}")
        return pd.DataFrame()  # Return empty DataFrame in case of failure

# Apply the function to fetch and merge fundamental data for MSFT
fundamental_data = fetch_fundamentals('MSFT')

# Displaying the first few rows of the fundamental dataset
fundamental_data


C:\Users\Administrator\AppData\Local\Temp\ipykernel_24060\1286849980.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fundamentals.fillna(method='backfill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24060\1286849980.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fundamentals.fillna(method='backfill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24060\1286849980.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fundamentals.fillna(method='ffill', inplace=True)


Free Cash Flow Growth Rate: 0.15863095238095237


,Beta,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,Total Capitalization,Total Equity Gross Minority Interest,Stockholders Equity,Gains Losses Not Affecting Retained Earnings,Other Equity Adjustments,Retained Earnings,Capital Stock,Common Stock,Total Liabilities Net Minority Interest,Total Non Current Liabilities Net Minority Interest,Other Non Current Liabilities,Tradeand Other Payables Non Current,Non Current Deferred Liabilities,Non Current Deferred Revenue,Non Current Deferred Taxes Liabilities,Long Term Debt And Capital Lease Obligation,Long Term Capital Lease Obligation,Long Term Debt,Current Liabilities,Other Current Liabilities,Current Deferred Liabilities,Current Deferred Revenue,Current Debt And Capital Lease Obligation,Current Debt,Pensionand Other Post Retirement Benefit Plans Current,Payables And Accrued Expenses,Payables,Total Tax Payable,Income Tax Payable,Accounts Payable,Total Assets,Total Non Current Assets,Other Non Current Assets,Investments And Advances,Long Term Equity Investment,Goodwill And Other Intangible Assets,Other Intangible Assets,Goodwill,Net PPE,Accumulated Depreciation,Gross PPE,Leases,Other Properties,Machinery Furniture Equipment,Buildings And Improvements,Land And Improvements,Properties,Current Assets,Other Current Assets,Hedging Assets Current,Inventory,Finished Goods,Work In Process,Raw Materials,Receivables,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Equivalents,Cash Financial,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Issuance Of Capital Stock,Capital Expenditure,End Cash Position,Beginning Cash Position,Effect Of Exchange Rate Changes,Changes In Cash,Financing Cash Flow,Cash Flow From Continuing Financing Activities,Net Other Financing Charges,Cash Dividends Paid,Common Stock Dividend Paid,Net Common Stock Issuance,Common Stock Payments,Common Stock Issuance,Net Issuance Payments Of Debt,Net Short Term Debt Issuance,Net Long Term Debt Issuance,Long Term Debt Payments,Long Term Debt Issuance,Investing Cash Flow,Cash Flow From Continuing Investing Activities,Net Other Investing Changes,Net Investment Purchase And Sale,Sale Of Investment,Purchase Of Investment,Net Business Purchase And Sale,Purchase Of Business,Net PPE Purchase And Sale,Purchase Of PPE,Operating Cash Flow,Cash Flow From Continuing Operating Activities,Change In Working Capital,Change In Other Working Capital,Change In Other Current Liabilities,Change In Other Current Assets,Change In Payables And Accrued Expense,Change In Payable,Change In Account Payable,Change In Inventory,Change In Receivables,Changes In Account Receivables,Stock Based Compensation,Deferred Tax,Deferred Income Tax,Depreciation Amortization Depletion,Depreciation And Amortization,Depreciation,Operating Gains Losses,Gain Loss On Investment Securities,Net Income From Continuing Operations
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-06-30,0.89,7.432000e+09,7.432000e+09,1.253300e+10,5.996500e+10,1.289710e+11,2.534600e+11,8.010800e+10,1.289710e+11,1.272800e+10,2.062230e+11,2.482130e+11,2.062230e+11,2.062230e+11,-6.343000e+09,-6.343000e+09,1.188480e+11,9.371800e+10,9.371800e+10,2.057530e+11,1.016040e+11,1.798100e+10,2.556000e+10,3.345000e+09,2.912000e+09,433000000.0,5.471800e+10,1.272800e+10,4.199000e+10,1.041490e+11,1.474500e+10,5.090100e+10,5.090100e+10,5.247000e+09,5.247000e+09,1.100900e+10,2.224700e+10,2.224700e+10,4.152000e+09,4.152000e+09,1.809500e+10,4.119760e+11,2.277190e+11,3.060100e+10,9.879000e+09,9.879000e+09,7.725200e+10,9.366000e+09,6.788600e+10,1.099870e+11,-6.825100e+10,1.782380e+11,8.537000e+09,1.434600e+10,8.120700e+10,6.846500e+10,5.68

In [501]:
def fetch_financial_data(ticker='SPY', start_year=1993, end_year=None, interval='1d', calculate_indicators=False, include_fundamentals=False, export_csv=False, csv_file=None,):
    """
    Fetches data for a specified ticker from Yahoo Finance from the given start year to the current year or specified end year at specified intervals.
    
    Parameters:
        ticker (str): The ticker symbol for the asset. Defaults to 'SPY'.
        start_year (int): The year from which to start fetching the data. Defaults to 1993.
        end_year (int): The last year for which to fetch the data. Defaults to the current year if None.
        interval (str): The data interval ('1d' for daily, '1wk' for weekly, '1mo' for monthly, '1h' for hourly).
        export_csv (bool): Whether to export the data to a CSV file. Defaults to False.
        csv_file (str): The path of the CSV file to export the data to. Automatically determined if None.
        calculate_indicators (bool): Flag to calculate technical indicators.
        include_fundamentals (bool): Flag to include fundamental data.

    Returns:
        DataFrame: DataFrame containing the requested financial data.
    """
    # Adjust for hourly data to limit to the last 730 days
    if interval == '1h':
        start_date = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')
    else:
        start_date = f'{start_year}-01-01'
    
    if end_year is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    else:
        end_date = f"{end_year}-12-31"
    
    if csv_file is None:
        csv_file = f'{ticker}_{interval}_data_{start_date}_to_{end_date}.csv'
    
    data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)
    
    if not data.empty:
        if calculate_indicators:
            # Here you would call your indicator functions on the `data` DataFrame
            data = bollinger_bands(data)
            data = macd(data)
            data = rsi(data)
            data = woodie_pivots(data)
            data = obv(data)
            data = atr(data)
            data = stochastic_oscillator(data)

            # Non-stationary data processing
            data = calculate_price_differences(data, 'Close')  # Calculate price differences
            data = calculate_log_returns(data, 'Close')  # Calculate log returns for the 'Close' column
            data = calculate_volume_changes(data, 'Volume')  # Calculate volume changes
            # Handling NaN values by forward filling then dropping rows with NaN values
            data.ffill(inplace=True)
            data.dropna(inplace=True)
       # Fetch and include fundamental data
        # if include_fundamentals:
        #     fundamental_data = fetch_fundamentals(ticker)
        #     data = pd.concat([data, fundamental_data], axis=1)
    return data

In [503]:
MSFT = fetch_financial_data('MSFT', interval="1d", calculate_indicators=True, include_fundamentals=True)

MSFT.head()

Free Cash Flow Growth Rate: 0.15863095238095237


C:\Users\Administrator\AppData\Local\Temp\ipykernel_24060\1286849980.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fundamentals.fillna(method='backfill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24060\1286849980.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fundamentals.fillna(method='backfill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24060\1286849980.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fundamentals.fillna(method='ffill', inplace=True)


,Open,High,Low,Close,Adj Close,Volume,Bollinger_High,Bollinger_Low,MACD,Signal,RSI,Pivot,R1,S1,R2,S2,R3,S3,R4,S4,OBV,ATR,%K,%D,Close_diff,Close_log_return,Volume_changes,Beta,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,Total Capitalization,Total Equity Gross Minority Interest,Stockholders Equity,Gains Losses Not Affecting Retained Earnings,Other Equity Adjustments,Retained Earnings,Capital Stock,Common Stock,Total Liabilities Net Minority Interest,Total Non Current Liabilities Net Minority Interest,Other Non Current Liabilities,Tradeand Other Payables Non Current,Non Current Deferred Liabilities,Non Current Deferred Revenue,Non Current Deferred Taxes Liabilities,Long Term Debt And Capital Lease Obligation,Long Term Capital Lease Obligation,Long Term Debt,Current Liabilities,Other Current Liabilities,Current Deferred Liabilities,Current Deferred Revenue,Current Debt And Capital Lease Obligation,Current Debt,Pensionand Other Post Retirement Benefit Plans Current,Payables And Accrued Expenses,Payables,Total Tax Payable,Income Tax Payable,Accounts Payable,Total Assets,Total Non Current Assets,Other Non Current Assets,Investments And Advances,Long Term Equity Investment,Goodwill And Other Intangible Assets,Other Intangible Assets,Goodwill,Net PPE,Accumulated Depreciation,Gross PPE,Leases,Other Properties,Machinery Furniture Equipment,Buildings And Improvements,Land And Improvements,Properties,Current Assets,Other Current Assets,Hedging Assets Current,Inventory,Finished Goods,Work In Process,Raw Materials,Receivables,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Equivalents,Cash Financial,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Issuance Of Capital Stock,Capital Expenditure,End Cash Position,Beginning Cash Position,Effect Of Exchange Rate Changes,Changes In Cash,Financing Cash Flow,Cash Flow From Continuing Financing Activities,Net Other Financing Charges,Cash Dividends Paid,Common Stock Dividend Paid,Net Common Stock Issuance,Common Stock Payments,Common Stock Issuance,Net Issuance Payments Of Debt,Net Short Term Debt Issuance,Net Long Term Debt Issuance,Long Term Debt Payments,Long Term Debt Issuance,Investing Cash Flow,Cash Flow From Continuing Investing Activities,Net Other Investing Changes,Net Investment Purchase And Sale,Sale Of Investment,Purchase Of Investment,Net Business Purchase And Sale,Purchase Of Business,Net PPE Purchase And Sale,Purchase Of PPE,Operating Cash Flow,Cash Flow From Continuing Operating Activities,Change In Working Capital,Change In Other Working Capital,Change In Other Current Liabilities,Change In Other Current Assets,Change In Payables And Accrued Expense,Change In Payable,Change In Account Payable,Change In Inventory,Change In Receivables,Changes In Account Receivables,Stock Based Compensation,Deferred Tax,Deferred Income Tax,Depreciation Amortization Depletion,Depreciation And Amortization,Depreciation,Operating Gains Losses,Gain Loss On Investment Securities,Net Income From Continuing Operations
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993-01-29,2.734375,2.750000,2.679688,2.703125,1.672747,39424000,2.864005,2.658652,0.008243,0.021679,43.908854,2.708984,2.738281,2.667969,2.779297,2.638672,2.808593,2.597657,2.919921,2.498048,-96377600,0.077846,18.918785,17.355565,-0.015625,-0.005764,-37606400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# Macro Data

In [337]:
def calculate_cashflow_growth_rate(free_cash_flows):
    return free_cash_flows.pct_change().mean()

def project_future_free_cash_flows(last_cash_flow, growth_rate, years):
    return [last_cash_flow * (1 + growth_rate) ** i for i in range(1, years + 1)]


def calculate_terminal_value(last_cash_flow, growth_rate, required_rate, years):
    return last_cash_flow * (1 + growth_rate) / (required_rate - growth_rate) / (1 + required_rate) ** years

def calculate_fair_value(discounted_cash_flows, terminal_value, outstanding_shares):
    total_present_value = sum(discounted_cash_flows) + terminal_value
    return total_present_value / outstanding_shares

def get_cost_of_equity(risk_free_rate, beta, market_return):
    return risk_free_rate + beta * (market_return - risk_free_rate)

def get_cost_of_debt(interest_rate, tax_rate):
    return interest_rate * (1 - tax_rate)

def get_proportions(market_value_equity, market_value_debt):
    total_value = market_value_equity + market_value_debt
    return market_value_equity / total_value, market_value_debt / total_value

def calculate_wacc(cost_of_equity, cost_of_debt, equity_proportion, debt_proportion, tax_rate):
    wacc = (cost_of_equity * equity_proportion) + ((1 - tax_rate) * cost_of_debt * debt_proportion)
    return wacc

def calculate_intrinsic_value(dividend_data, discount_rate):
    intrinsic_value = 0
    for year, dividend in enumerate(dividend_data, start=1):
        if year <= 5:
            growth_rate = 0.05
        elif 5 < year <= 10:
            growth_rate = 0.03
        else:
            growth_rate = 0.01
        intrinsic_value += dividend / ((1 + discount_rate) ** year)
    return intrinsic_value

def calculate_cost_of_equity(beta, risk_free_rate, market_return):
    """
    Calculate the cost of equity using the CAPM formula.
    
    :param beta: Beta of the stock
    :param risk_free_rate: Risk-free rate
    :param market_return: Expected market return
    :return: Cost of equity
    """
    return risk_free_rate + beta * (market_return - risk_free_rate)


# Valuation Models for Each Asset Class:

In [338]:
def dcf_valuation(cash_flows, discount_rate):
    """
    Calculate the present value of cash flows using the discounted cash flow (DCF) method.
    
    Args:
    - cash_flows (list): List of projected cash flows.
    - discount_rate (float): Discount rate (required rate of return).
    
    Returns:
    - float: Present value of the cash flows.
    """
    dcf_value = sum(cf / (1 + discount_rate)**n for n, cf in enumerate(cash_flows, start=1))
    return dcf_value

def calculate_expected_return(risk_free_rate, beta, market_return, market_risk_premium):
    """
    Calculate the expected return of an asset using the Capital Asset Pricing Model (CAPM).
    
    Args:
    - risk_free_rate (float): Risk-free rate (e.g., yield on Treasury bills).
    - beta (float): Beta coefficient of the asset.
    - market_return (float): Expected return of the market portfolio.
    - market_risk_premium (float): Market risk premium.
    
    Returns:
    - float: Expected return of the asset.
    """
    expected_return = risk_free_rate + beta * market_risk_premium
    return expected_return

def three_stage_dividend_discount_model(symbol, discount_rate):
    dividend_data = fetch_dividend_data(symbol)
    intrinsic_value = calculate_intrinsic_value(dividend_data, discount_rate)
    return intrinsic_value


def residual_income_model(net_income, equity, required_return):
    """
    Calculate the value of equity using the Residual Income Model.
    
    Args:
    - net_income (float): Net income of the company.
    - equity (float): Book value of equity.
    - required_return (float): Required rate of return on equity.
    
    Returns:
    - float: Estimated value of equity using the Residual Income Model.
    """
    # Calculate the present value of expected future residual income
    residual_income = net_income - (required_return * equity)
    
    # Value of equity is the book value of equity plus the present value of expected future residual income
    equity_value = equity + residual_income
    
    return equity_value



In [375]:
# Assuming free_cash_flows is a pandas Series containing free cash flow data
growth_rate = calculate_cashflow_growth_rate(free_cash_flows)
print("Free Cash Flow Growth Rate:", growth_rate)


NameError: name 'free_cash_flows' is not defined